In [10]:
import pandas as pd
from scipy.spatial.distance import euclidean
from process import *
from utils import *
from model import *

In [11]:
#prendo i dati relativi al dataset
device='cpu'
layer=3
adj, adj_i, features, labels, idx_train, idx_val, idx_test, num_features, num_labels, g = full_load_data_custom('chameleon','splits/chameleon_split_0.6_0.2_0.npz')
features = features.to(device)

# processo la matrice di adiacenza del grafo per darlo in input alla rete
adj = adj.to(device)
adj_i = adj_i.to(device)
list_mat = []
list_mat.append(features)
no_loop_mat = features
loop_mat = features

for ii in range(layer):
    no_loop_mat = torch.spmm(adj, no_loop_mat)
    loop_mat = torch.spmm(adj_i, loop_mat)
    list_mat.append(no_loop_mat)
    list_mat.append(loop_mat)

In [12]:
layer_norm=bool(1)
# Define the path to the saved model checkpoint
PATH = "chameleon\chameleon25.pt"
GNNchameleon=FSGNN(nfeat=num_features,
                nlayers=len(list_mat),
                nhidden=64,
                nclass=num_labels,
                dropout=0.5).to(device)
GNNchameleon.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [13]:
#prendo l'embedding dalla rete
GNNchameleon.eval()
out = GNNchameleon.emb(list_mat, layer_norm)
torch.save(out,'chameleon\emb_chameleon25.pt')


In [14]:
out.shape

torch.Size([2277, 448])

In [15]:
#trasformo la matrice di adiacenza in grafo
G = nx.Graph(g)
adj_mat=pd.DataFrame(columns=['target','source'])
adj_mat = nx.to_pandas_edgelist(G)
adj_mat = adj_mat[adj_mat['source']!=adj_mat['target']]
adj_mat

,source,target,weight
0,0,1161,1
1,0,1667,1
2,0,1991,1
3,0,2130,1
4,0,2156,1
...,...,...,...
31415,2260,2263,1
31416,2260,2270,1
31417,2263,2266,1
31418,2263,2270,1


In [16]:
#definizione distanza manhattan
import numpy as np

def manhattan_distance(arr1, arr2):
    """
    Calculates the Manhattan distance between two arrays of the same length.
    """
    return np.sum(np.abs(arr1 - arr2))

In [17]:
#aggiungo i pesi ottenuti dall'embedding
df_emb = pd.DataFrame(out.detach().numpy(), columns=['feature_' + str(x) for x in range(448)])
adj_mat['weight'] = adj_mat.apply(lambda x:
                                    manhattan_distance(df_emb.loc[x['target']].values,
                                    df_emb.loc[x['source']].values),
                                    axis=1)
adj_mat.to_csv('chameleon\chameleon_weights25.csv', index = False)

In [ ]:
#label della rete 
pd.DataFrame(labels.numpy(),columns=['label']).to_csv('chameleon\chameleon_labels.csv', index = False)